In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score


In [160]:
sample_submission = pd.read_csv('Sample_submission.csv')
X_train = pd.read_csv('train.csv')
y_train = X_train.Attrition
X_train.drop(['Attrition','EmployeeNumber','Behaviour'], axis=1, inplace=True)
#X_train.drop(['Attrition'], axis=1, inplace=True)
X_test = pd.read_csv('test.csv')
X_test.drop(['EmployeeNumber','Behaviour'], axis=1, inplace=True)

In [161]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

In [162]:
X_train = pd.get_dummies(X_train, prefix_sep=" ",columns=object_cols)
X_test = pd.get_dummies(X_test, prefix_sep=" ",columns=object_cols)

In [163]:
X_train.drop(['BusinessTravel Travel_Rarely','Department Sales','EducationField Technical Degree', 'Gender Male','JobRole Sales Representative','MaritalStatus Single','OverTime Yes'], axis=1, inplace=True)
X_test.drop(['BusinessTravel Travel_Rarely','Department Sales','EducationField Technical Degree', 'Gender Male','JobRole Sales Representative','MaritalStatus Single','OverTime Yes'], axis=1, inplace=True)

In [170]:
model = LogisticRegression(C=0.9,solver='lbfgs',max_iter=50000,penalty='l2',random_state=50,fit_intercept=False)
model.fit(X_train, y_train)

LogisticRegression(C=0.9, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=50000,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=50, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [171]:
a=model.predict_proba(X_test)
b=a[:,1]
ss=sample_submission.drop('Attrition',axis=1)
ss['Attrition']=b
ss.to_csv('submissionnew.csv',index=False)
ss.tail()

,Id,Attrition
465,466,0.306033
466,467,0.716136
467,468,0.048903
468,469,0.189155
469,470,0.013433


In [ ]:
conda install -c glemaitre imbalanced-learn

In [145]:
# Create first pipeline for base without reducing features.
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


pipe = Pipeline([('classifier' , RandomForestClassifier())])
# pipe = Pipeline([('classifier', RandomForestClassifier())])

# Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : [ 'l1','l2'],
    'classifier__C' : np.logspace(-25, 25, 20),
    'classifier__solver' : ['liblinear'],
    'classifier__max_iter' : list(range(100,50000,500000)),
    'classifier__random_state' : list(range(1,10,1))},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,150,10)),
    'classifier__max_features' : list(range(6,32,5))}
]

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(X_train, y_train)

Fitting 5 folds for each of 444 candidates, totalling 2220 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 1191 tasks      | elapsed:   22.0s


KeyboardInterrupt: 

In [ ]:
a=best_clf.predict_proba(X_test)
b=a[:,1]
ss=sample_submission.drop('Attrition',axis=1)
ss['Attrition']=b
ss.to_csv('submissionnew.csv',index=False)
ss.tail()